In [31]:
import unidecode
from nltk.translate.bleu_score import sentence_bleu
import random as rd

play_file_infixes = ['othello', 'antony-and-cleopatra', 'asyoulikeit', 
                     'errors', 'hamlet', 'henryv', 'juliuscaesar', 'lear', 'macbeth', 
                     'merchant', 'msnd', 'muchado', 'richardiii', 'romeojuliet', 
                     'shrew', 'tempest', 'twelfthnight']
play_file_infixes = ['othello']

agg_original_tuples = []
agg_modern_tuples = []
path = "data/shakespeare/data/align/plays/merged/"

for infix in play_file_infixes:
    modern_tuples = unidecode.unidecode(open(path + infix + "_modern.snt.aligned").read()).split("\n")
    original_tuples = unidecode.unidecode(open(path + infix + "_original.snt.aligned").read()).split("\n")
    agg_original_tuples.extend(original_tuples)
    agg_modern_tuples.extend(modern_tuples)

In [32]:
print("{}\n{}\n".format(agg_original_tuples[0], agg_modern_tuples[0]))
#print("{}\n{}\n".format(agg_original_tuples[5000], agg_modern_tuples[5000]))
# print("{}\n{}\n".format(agg_original_tuples[9000], agg_modern_tuples[9000]))
# print("{}\n{}\n".format(agg_original_tuples[18000], agg_modern_tuples[18000]))


What's the matter, lieutenant?
What's the matter, lieutenant?



In [33]:
# in-class = original = '1'
# raw -> (raw text, label)
import numpy as np

style1_np = np.array(agg_original_tuples)
style1_labels = np.array(['1' for x in range(len(agg_original_tuples))])
X = np.vstack((style1_np, style1_labels))

style2_np = np.array(agg_modern_tuples)
style2_labels = np.array(['0' for x in range(len(agg_modern_tuples))])
X2 = np.vstack((style2_np, style2_labels))

X = np.hstack((X,X2))
raw = np.transpose(X)

idx_col = [str(x) for x in range(len(raw))]
idx_col = np.array(idx_col).reshape(-1,1)
print(idx_col.shape)
print(raw.shape)
raw = np.hstack((idx_col, raw))
raw.shape

(3710, 1)
(3710, 2)


(3710, 3)

In [34]:
raw[2000]

array(['2000', "CASSIO  She's awake, sir.", '0'], dtype='<U497')

In [35]:
from sklearn.feature_extraction import DictVectorizer
import spacy
import scipy.sparse
nlp = spacy.load('en_core_web_sm')


vectorizer = DictVectorizer()
features = []

# feature engineering
    # construct index column
idx_col = np.array([raw[:,0]])
idx_col = idx_col.astype('f', copy=True)

#bleu: 0-1, 1 is perfect score

bleu = sentence_bleu([sen2],sen1)

raw_text_col = 1
LABEL_COL = -1

for row_idx,_ in enumerate(raw):
    feature_dict = {}
    pos_dict = {'NOUN_count': 0, 'ADV_count': 0, 'VERB_count': 0, 
                'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0, 'bleu': 0}
    txt = str(raw[row_idx][raw_text_col])
    
    feature_dict['sent_len'] = len(txt)
    try:
        doc = nlp(txt)
    except IndexError as e:
        print(e)
        print("offending message [{}], len: {}, row idx: {}".format(txt, len(txt), row_idx))

    for token in doc:
        pos = token.pos_
        pos_dict[pos+'_count'] = pos_dict.get(pos+'_count', 0) + 1

    if pos_dict['NOUN_count'] == 0 or pos_dict['ADJ_count'] == 0:
        pos_dict['adj_noun_ratio'] = 0
    else:
        pos_dict['adj_noun_ratio'] = pos_dict['NOUN_count'] / pos_dict['ADJ_count'] 

    if pos_dict['VERB_count'] == 0 or pos_dict['ADV_count'] == 0:
        pos_dict['adv_verb_ratio'] = 0
    else:
        pos_dict['adv_verb_ratio'] = pos_dict['VERB_count'] / pos_dict['ADV_count'] 
        
    feature_dict.update(pos_dict)
    
    if row_idx % 1000 == 0:
        print("{} gold: {} {}\n".format(txt, raw[row_idx][LABEL_COL], feature_dict) )
        
    features.append(feature_dict)

labels_col = np.array([raw[:,LABEL_COL].astype(float)])

X = vectorizer.fit_transform(features)
print(type(X))
print(X.shape)
print(idx_col.T.shape)
X = scipy.sparse.hstack(( idx_col.T, X, labels_col.T ))
X = X.todense()
# X = np.transpose(X)
# X = X.astype('f', copy=True)
# print(X[0:5])
# print(X[3700:3705])

What's the matter, lieutenant? gold: 1 {'sent_len': 30, 'NOUN_count': 3, 'ADV_count': 0, 'VERB_count': 1, 'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0, 'DET_count': 1, 'PUNCT_count': 2}

Therefore, good Emilia, Give me my nightly wearing, and adieu. gold: 1 {'sent_len': 62, 'NOUN_count': 1, 'ADV_count': 2, 'VERB_count': 2, 'ADJ_count': 2, 'adv_verb_ratio': 1.0, 'adj_noun_ratio': 0.5, 'PUNCT_count': 4, 'PROPN_count': 1, 'PRON_count': 1, 'CCONJ_count': 1}

list index out of range
offending message [], len: 0, row idx: 1854
CASSIO  She's awake, sir. gold: 0 {'sent_len': 25, 'NOUN_count': 1, 'ADV_count': 0, 'VERB_count': 1, 'ADJ_count': 1, 'adv_verb_ratio': 0, 'adj_noun_ratio': 1.0, 'PROPN_count': 1, 'SPACE_count': 1, 'PRON_count': 1, 'PUNCT_count': 2}

Who's shouting? gold: 0 {'sent_len': 15, 'NOUN_count': 1, 'ADV_count': 0, 'VERB_count': 2, 'ADJ_count': 0, 'adv_verb_ratio': 0, 'adj_noun_ratio': 0, 'PUNCT_count': 1}

list index out of range
offending message [], len: 0, row id

In [36]:
raw[1853][raw_text_col]

'It were an honest action to say So to the Moor.'

In [37]:
X[1:2]

matrix([[ 1.,  0.,  1.,  0.,  0.,  2.,  0.,  3.,  0.,  0.,  1.,  0.,  1.,
          0.,  2.,  0.,  0.,  0., 42.,  1.]])

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import time
import math
###############
# training ####
###############
# test breakout: 3500, 210
train_val_split_delim = math.floor(len(X) * .8)
val_test_split_delim = math.floor(len(X) * .9)
print("total rows: {} train val split delim: {} val test split delim: {}".format(len(X), train_val_split_delim, val_test_split_delim))
# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim
last_feat_col = X.shape[1] -2
X = np.random.permutation(X)

X_train = X[0:train_val_split_delim,1:last_feat_col ]
Y_train = X[0:train_val_split_delim,LABEL_COL]

X_val = X[train_val_split_delim:val_test_split_delim,1:last_feat_col ]
Y_val = X[train_val_split_delim:val_test_split_delim,LABEL_COL]

X_test = X[val_test_split_delim:,1:last_feat_col ]
Y_test = X[val_test_split_delim:,LABEL_COL]


#mdl = LogisticRegression(random_state=123)
mdl = MultinomialNB()
mdl.fit(X_train, Y_train)

np.set_printoptions(suppress=True)


total rows: 3710 train val split delim: 2968 val test split delim: 3339


In [39]:
mdl.classes_

array([0., 1.])

In [56]:
# evaluation ###
################
#second class is original

from sklearn.metrics import brier_score_loss

val_indices = X[train_val_split_delim:val_test_split_delim,0]

# def brier_score(raw, preds, val_indices, loss='SSE'):
#     if len(preds) != len(val_indices):
#         raise ArgumentError("weighted_eval: list length mismatch!")
        
#     results = []
    
#     for (idx, pred) in zip(val_indices, preds):
#         gold_label = int(raw[int(idx)][-1])
        
#         #pred_label = int(pred)
#         #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))

#         predicted_class = 0 if (pred[0] > pred[1]) else 1
        
#         if gold_label == pred_label:
#               results.append( (1 - pred[predicted_class])**2 )
#         else:
#               results.append( (pred[predicted_class])**2 )
#         print("{} | gold: {} predicted: {}".format(raw[idx][1], gold_label, pred ))
#     return sum(results)/len(preds)
    
#binary evaluation
# preds = mdl.predict(X_test)
# 
# test_indices = X[val_test_split_delim:,0]
# if len(preds) != len(val_indices):
#     raise ValueError("unmatched arrays")
    
# results = []
# for (idx, pred) in zip(val_indices,preds):
#     gold_label = int(raw[int(idx)][-1])
#     pred_label = int(pred)
#     #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))
    
#     if gold_label == pred_label:
#           results.append(1)
#     else:
#         results.append(0)
# results[:5]

preds = mdl.predict_proba(X_test)

y_true = [int(raw[int(idx)][-1]) for idx in val_indices]

brier = brier_score_loss(y_true, preds[:,0]) 
print("brier score:", brier)

#currently slightly worse than randomly permuting preds

brier score: 0.2776796938888296


In [ ]:
# test_len = len(preds)
# accuracy = sum(preds)/float(test_len)
# print("accuracy: {}".format(accuracy))

#othello accuracy: 0.7428571428571429. but maybe not permuted
#all: accuracy: 0.328237942884228

#pos, no bleu, 2 plays
# Mult NB: .519